In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [35]:
data = pd.read_csv("Churn_Modelling.csv")

In [36]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Estimated Salary  is our target feature 

In [37]:
data = data.drop(["RowNumber","CustomerId","Surname"],axis = 1)

In [38]:
# Encoder categorical variables
lb = LabelEncoder()
data["Gender"] = lb.fit_transform(data["Gender"])

In [39]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [40]:
## Applying one hot encoding to the Geography

ohe = OneHotEncoder()
df = ohe.fit_transform(data[["Geography"]]).toarray()

df = pd.DataFrame(df,columns=ohe.get_feature_names_out(["Geography"]))

df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [41]:
## concatenation of this data to the original data

data = pd.concat([data,df],axis = 1)

In [42]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [43]:
## Now we can remove the Geography feature in the dataset
data = data.drop("Geography",axis = 1)

In [44]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [45]:
f = data.drop("EstimatedSalary",axis = 1)
f.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [46]:
## Now do divided the dependent and indpendent features 

X = data.drop("EstimatedSalary",axis = 1)
y = data["EstimatedSalary"]

In [47]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0


In [48]:
y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [49]:
## Lets do the train test split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [50]:
## Applying standardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [51]:
## Lets store encoders and scaler in pickle file

with open("label_encoder_reg_gender.pkl",'wb') as f:
    pickle.dump(lb,f)

with open("one_hot_encoder_reg_geo.pkl",'wb') as f:
    pickle.dump(ohe,f)

with open("StandardScaler_reg.pkl",'wb') as f:
    pickle.dump(scaler,f)

# ANN Regressoin Problem Statment

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [53]:
model = Sequential([
    Dense(64,activation='relu',input_shape = (X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) # Output layer for regression 

]

)

# here in output layer we are not using any activation functions cause it is linear regression problem, by default we have 
# linear activation function that means whatever the output is gotten during the step 1 of neuron that is being passed as output without any changes

c:\Users\deran\Desktop\DL_Project_1\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
model.compile(optimizer='adam',loss = 'mean_absolute_error',metrics = ['mae'])

In [55]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
tensorboard_callback = TensorBoard(log_dir = log_dir,histogram_freq = 1)

In [57]:
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights = True)


In [58]:
history = model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs = 100,
    callbacks = [early_stopping_callback,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 100359.9453 - mae: 100359.9453 - val_loss: 98476.7656 - val_mae: 98476.7656
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 101207.0078 - mae: 101207.0078 - val_loss: 96669.9453 - val_mae: 96669.9453
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 98961.6953 - mae: 98961.6953 - val_loss: 92073.5625 - val_mae: 92073.5625
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 92688.4219 - mae: 92688.4219 - val_loss: 84589.7812 - val_mae: 84589.7812
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 84257.8203 - mae: 84257.8203 - val_loss: 75275.7500 - val_mae: 75275.7500
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 74707.0078 - mae: 74707.0078 - val_loss: 66034.8672 - val_mae: 66034.8672
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 65495.8516 - mae: 65495.8516 - val_loss: 58635.8984 - val_mae: 58635.8984
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/st

In [59]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [60]:
%tensorboard --logdir=regressionlogs/fit

Reusing TensorBoard on port 6008 (pid 8836), started 21:12:01 ago. (Use '!kill 8836' to kill it.)

In [61]:
test_loss,test_mae = model.evaluate(X_test,y_test)
print(f"Test MAE  : {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 51129.7344 - mae: 51129.7344
Test MAE  : 50252.87890625


In [62]:
model.save("regression_model.h5")